In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/donations/hag_county_2015-full.csv')

In [3]:
data_x1 = data.iloc[:,[4,6]]

X = data_x1.set_index('cbsa_code')

X.isnull().values.sum()

X = X[np.isfinite(X['cy_gratio'])]

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

range_clusters = [2,3,4,5,6,7,8]

for n_clusters in range_clusters:
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

For n_clusters = 2 The average silhouette_score is : 0.594363720525
For n_clusters = 3 The average silhouette_score is : 0.552671052673
For n_clusters = 4 The average silhouette_score is : 0.535421371304
For n_clusters = 5 The average silhouette_score is : 0.5220659461
For n_clusters = 6 The average silhouette_score is : 0.525284595426
For n_clusters = 7 The average silhouette_score is : 0.527183828595
For n_clusters = 8 The average silhouette_score is : 0.535806380022


In [4]:
data_x1 = data.iloc[:,[4,6]]
X = data_x1.set_index('cbsa_code')
X.isnull().values.sum()
X = X[np.isfinite(X['cy_gratio'])]

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


clust = KMeans(n_clusters=4, random_state = 13, n_jobs=-1)
clust.fit_predict(X_scaled)
label = clust.labels_
series = pd.Series(label)


In [5]:
X = X.reset_index()

X_clustered = pd.concat([X,series],axis=1)

In [6]:
X_clustered.rename(columns={0:'cluster_label'},inplace=True)

In [7]:
data = data[np.isfinite(data['cbsa_code'])]

In [8]:
data['cbsa_code'] = data['cbsa_code'].astype('str')
data['cy_gratio'] = data['cy_gratio'].astype('str')
data['key'] = data['cbsa_code'].map(str) + data['cy_gratio']

In [9]:
X_clustered['cbsa_code'] = X_clustered['cbsa_code'].astype('str')
X_clustered['cy_gratio'] = X_clustered['cy_gratio'].astype('str')
X_clustered['key'] = X_clustered['cbsa_code'].map(str) + X_clustered['cy_gratio']

In [10]:
data = data.set_index('key')
X_clustered = X_clustered.set_index('key')
data_cl = data.join(X_clustered,lsuffix='_d',rsuffix='_c')

In [11]:
data_cl['cy_gratio_d'] = data_cl['cy_gratio_d'].astype('float64')
data_cl.pivot_table(['cy_pop_2015', 'cy_gratio_d',
       'cy_avg_giving_by_itemizer', 'gvsa', 'god', 'cy_itemized_agi',
       'cy_amt_contributions'],['cluster_label'], aggfunc='mean')

,cy_amt_contributions,cy_avg_giving_by_itemizer,cy_gratio_d,cy_itemized_agi,cy_pop_2015,god,gvsa
cluster_label,,,,,,,
0.0,103988.674772,5.397374,0.036972,2.791705e+06,148342.817629,4347.200674,0.079599
1.0,87603.690476,11.709386,0.073217,1.074099e+06,72485.571429,507.719957,0.504567
2.0,70211.139024,7.073881,0.050730,1.417285e+06,91361.041463,2400.992866,0.196058
3.0,133387.576923,3.760210,0.025186,5.266210e+06,225513.155172,17916.598187,-0.126522


In [12]:
data_cl.to_csv('D:/UTD_JSOM/Spring 2018/Practicum/Market Analysis/four_cluster.csv')